In [1]:
from tensorflow.keras.datasets import mnist

from tensorflow.keras.layers import Input, Dense, Reshape, Flatten, BatchNormalization, LeakyReLU, Activation, Dropout, Conv2D
from tensorflow.keras import activations
from tensorflow.keras.models import Model, Sequential

from tensorflow.keras.losses import binary_crossentropy

from tensorflow.keras.optimizers import Adam
from matplotlib import pyplot as plt

import numpy as np, os, cv2

## Platform

In [3]:
colab = False

# Configurations

In [6]:
image_shape = (28, 28, 1)
input_noise_shape = (100,)
training_epochs = 20000

In [4]:
directory = '/content/drive/My Drive/Colab Notebooks' if colab else '.'

if colab:
    from google.colab import drive
    drive.mount('/content/drive')

trainedfile = 'model.h5'
checkpointfolder = 'mnist/models'
resultsfolder = 'mnist/results'

if not os.path.isdir(os.path.join(directory, checkpointfolder)):
    os.makedirs(os.path.join(directory, checkpointfolder))

if not os.path.isdir(os.path.join(directory, resultsfolder)):
    os.makedirs(os.path.join(directory, resultsfolder))

In [5]:
# constants
REAL=1
FAKE=0

# Load training data

In [6]:
def loader():
    
    (X_train, _), _ = mnist.load_data()    
    # X_train=X_train[:10]
    resized=[]
    for i in range(len(X_train)):
        resized.append(cv2.resize(X_train[i], image_shape[:2]))
    resized=np.array(resized)
    norm_X_train = (resized-128.0)/255.0    
    flatNorm_X_train = norm_X_train.reshape((*norm_X_train.shape, 1))

#     norm_X_train = np.loadtxt('/content/drive/My Drive/Colab Notebooks/mnist/test.in').reshape(*image_shape)
#     flatNorm_X_train = norm_X_train.reshape((*norm_X_train.shape, 1))
    print(f'Loaded {len(flatNorm_X_train)} images.')
    
    return flatNorm_X_train

# Generator architecture

In [7]:
def GENERATOR():    
    
    model = Sequential()
    
    model.add(Dense(256, input_shape=input_noise_shape))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum=0.8))
    
    model.add(Dense(256))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum=0.8))
    
    model.add(Dense(1024))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum=0.8))   
    
    model.add(Dense(np.prod(image_shape)))
    model.add(Activation(activations.tanh))
    model.add(Reshape(image_shape))
    
    model.summary()     
    
    
    noisy_input_end = Input(shape=input_noise_shape)    
    image_generation_end = model(noisy_input_end)
    
    return Model(noisy_input_end, image_generation_end)

# Discriminator architecture

In [8]:
def DISCRIMINATOR():    
    
    model= Sequential()
    
    model.add(Flatten(input_shape=image_shape))
    
    model.add(Dense(512))
    model.add(LeakyReLU(alpha=0.2))
    
    model.add(Dense(256))
    model.add(LeakyReLU(alpha=0.2))
    
    model.add(Dense(1))
    model.add(Activation(activations.sigmoid))
    
    model.summary()    
    
    
    image_discriminator_input_end = Input(shape=image_shape)
    image_discriminator = model(image_discriminator_input_end)
    
    return Model(image_discriminator_input_end, image_discriminator)

# Training code

In [9]:
def train(epochs, generator, discriminator, combined, flatNorm_X_train, batch_size = 1024):    
    
    
    for epoch in range(epochs):
        
        # training discriminator
        
        real_imgs_idx = np.random.randint(0, len(flatNorm_X_train), batch_size//2)
        real_imgs = flatNorm_X_train[real_imgs_idx]
        
        random_noise_input = np.random.normal(0, 1, (batch_size//2, *input_noise_shape))        
        fake_imgs = generator.predict(random_noise_input)
        
        real_image_loss = discriminator.train_on_batch(real_imgs, np.array([REAL]*(batch_size//2)))
        fake_image_loss = discriminator.train_on_batch(fake_imgs, np.array([FAKE]*(batch_size//2)))
        
        discriminator_loss = 0.5 * np.add(real_image_loss, fake_image_loss)
        
        
        #training generator, tricking the discriminator        
        random_noise_input = np.random.normal(0, 1, (batch_size, *input_noise_shape))        
        generator_loss = combined.train_on_batch(random_noise_input, np.array([REAL]*batch_size))

        print ("%d [D loss: %f,D acc.: %.2f%%] [G loss: %f,G acc.: %.2f%%]" % (epoch, discriminator_loss[0], 100*discriminator_loss[1], generator_loss[0], 100*generator_loss[1]))

# Initialization

In [10]:
if __name__ == "__main__":
    
    optimizer = Adam(0.0002, 0.5)
    
    flatNorm_X_train = loader()
    
    discriminator = DISCRIMINATOR()
    discriminator.compile(loss=binary_crossentropy, optimizer=optimizer, metrics=['accuracy'])
    
    
    generator = GENERATOR()
    generator.compile(loss=binary_crossentropy, optimizer=optimizer)
    
    
    combined_noisy_input_end = Input(shape=input_noise_shape)
    generator_tailend = generator(combined_noisy_input_end)
    discriminator.trainable = False
    discriminator_tailend = discriminator(generator_tailend)
    
    combined = Model(combined_noisy_input_end, discriminator_tailend)
    combined.compile(loss=binary_crossentropy, optimizer=optimizer, metrics=['accuracy'])

11493376/11490434 [==============================] - 0s 0us/step
Loaded 60000 images.
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 784)               0         
_________________________________________________________________
dense (Dense)                (None, 512)               401920    
_________________________________________________________________
leaky_re_lu (LeakyReLU)      (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               131328    
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 256)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 257       
____________________________________

# Training

In [11]:
if __name__ == '__main__':
    
    train(training_epochs, generator, discriminator, combined, flatNorm_X_train)
    generator.save(os.path.join(directory, checkpointfolder, trainedfile))

Streaming output truncated to the last 5000 lines.
15000 [D loss: 0.136660,D acc.: 94.92%] [G loss: 5.881542,G acc.: 0.00%]
15001 [D loss: 0.090727,D acc.: 96.68%] [G loss: 5.937862,G acc.: 0.59%]
15002 [D loss: 0.083433,D acc.: 98.34%] [G loss: 7.078507,G acc.: 0.00%]
15003 [D loss: 0.088172,D acc.: 96.68%] [G loss: 5.768284,G acc.: 0.00%]
15004 [D loss: 0.045103,D acc.: 99.12%] [G loss: 6.537792,G acc.: 0.00%]
15005 [D loss: 0.064023,D acc.: 97.95%] [G loss: 4.920656,G acc.: 0.20%]
15006 [D loss: 0.065549,D acc.: 98.24%] [G loss: 5.188593,G acc.: 0.10%]
15007 [D loss: 0.096297,D acc.: 96.19%] [G loss: 6.660308,G acc.: 0.00%]
15008 [D loss: 0.077347,D acc.: 97.56%] [G loss: 7.296814,G acc.: 0.00%]
15009 [D loss: 0.092857,D acc.: 96.19%] [G loss: 5.485878,G acc.: 0.29%]
15010 [D loss: 0.113978,D acc.: 96.39%] [G loss: 6.019227,G acc.: 0.00%]
15011 [D loss: 0.135518,D acc.: 95.02%] [G loss: 5.756227,G acc.: 0.00%]
15012 [D loss: 0.097835,D acc.: 96.48%] [G loss: 6.770316,G acc.: 0.00%]


NameError: name 'generated_images' is not defined

# Prediction

In [7]:
from tensorflow import keras
import numpy as np, cv2, os

generator=keras.models.load_model(os.path.join(directory, checkpointfolder, trainedfile))

if colab:
    from google.colab.patches import cv2_imshow

    
num_of_image_to_generate = 100
generated_images=generator(np.random.uniform(-2, 0, (num_of_image_to_generate, *input_noise_shape)))

for i, image in enumerate(generated_images):    
    resized=cv2.resize(image.numpy(), (64, 64), interpolation = cv2.INTER_AREA)*255.0 + 128.0
    if colab:
        cv2_imshow(resized)
    if cv2.imwrite(os.path.join(directory, resultsfolder, f'mnist-{i}.png'), resized):
        print(f'Saved as {os.path.join(directory, resultsfolder, f"mnist-{i}.png")}')            

Saved as ./mnist/results/mnist-0.png
Saved as ./mnist/results/mnist-1.png
Saved as ./mnist/results/mnist-2.png
Saved as ./mnist/results/mnist-3.png
Saved as ./mnist/results/mnist-4.png
Saved as ./mnist/results/mnist-5.png
Saved as ./mnist/results/mnist-6.png
Saved as ./mnist/results/mnist-7.png
Saved as ./mnist/results/mnist-8.png
Saved as ./mnist/results/mnist-9.png
Saved as ./mnist/results/mnist-10.png
Saved as ./mnist/results/mnist-11.png
Saved as ./mnist/results/mnist-12.png
Saved as ./mnist/results/mnist-13.png
Saved as ./mnist/results/mnist-14.png
Saved as ./mnist/results/mnist-15.png
Saved as ./mnist/results/mnist-16.png
Saved as ./mnist/results/mnist-17.png
Saved as ./mnist/results/mnist-18.png
Saved as ./mnist/results/mnist-19.png
Saved as ./mnist/results/mnist-20.png
Saved as ./mnist/results/mnist-21.png
Saved as ./mnist/results/mnist-22.png
Saved as ./mnist/results/mnist-23.png
Saved as ./mnist/results/mnist-24.png
Saved as ./mnist/results/mnist-25.png
Saved as ./mnist/resul